In [15]:
import requests
import pprint
import xml.etree.ElementTree as ET
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta

In [22]:
valid_opi_list = []
data = pd.read_csv('./observation_point_info.csv',encoding="CP949")
OPI_df = pd.DataFrame(data)
observation_point_num_list = [90,93,95,98,99,100,101,102,104,105,106,108,112,114,115,119,121,127,129,130,131,133,135,136,137,138,140,143,146,152,155,156,159,162,165,168,169,170,172,174,177,184,185,188,189,192,201,202,203,211,212,216,217,221,226,232,235,236,238,239,243,244,245,247,248,251,252,253,254,255,257,258,259,260,261,262,263,264,266,268,271,272,273,276,277,278,279,281,283,284,285,288,289,294,295]

opi2_df = OPI_df[OPI_df['위도']<37]
final_opi_df = opi2_df[opi2_df['위도']>36]
opi_point_list = sorted(set(final_opi_df['지점']))

opi_middle_list = []
for point in opi_point_list:
    if point in observation_point_num_list:
        opi_middle_list.append(str(point))

In [23]:
opi_middle_list

['127',
 '129',
 '130',
 '131',
 '133',
 '135',
 '136',
 '137',
 '138',
 '140',
 '177',
 '226',
 '232',
 '235',
 '236',
 '238',
 '239',
 '271',
 '272',
 '273',
 '276',
 '277',
 '278',
 '279']

In [24]:
import requests
import pprint
import xml.etree.ElementTree as ET
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime, timedelta


def date_change(date):
    nowdate = datetime.strptime(date, "%Y-%m-%d")

    nextYear = nowdate.replace(year=nowdate.year + 1)
    endDate = (nextYear - timedelta(days=1)).strftime("%Y%m%d")
    startDate = nowdate.strftime("%Y%m%d")

    return startDate, endDate


def parse_month(year, month, stnid, stnnm, m_temp, m_gtemp):

    try:

        # 지점 이름, 지점 아이디

        return {
            "년도": year,
            "월": month,
            "지점 번호": stnid,
            "지점명": stnnm,
            "월평균 기온": m_temp,
            "월평균 지면온도": m_gtemp,
        }

    except AttributeError as e:
        return {
            "년도": None,
            "월": None,
            "지점 번호": None,
            "지점명": None,
            "월평균 기온": None,
            "월평균 지면온도": None,
        }


def mean_cul(item, result_m):

    STNID = item.find("stnId").get_text()
    STNNM = item.find("stnNm").get_text()
    try:
        temp = float(item.find("avgTa").get_text())
        ground_temp = float(item.find("avgTs").get_text())
    except ValueError:
        temp = 0
        ground_temp = 0
        if "error" in result_m:
            result_m["error"] = result_m["error"] + 1
        else:
            result_m["error"] = 1

    result_m["stnid"] = STNID
    result_m["stnnm"] = STNNM

    if "t_total" in result_m:
        result_m["t_total"] = result_m["t_total"] + temp
        result_m["gt_total"] = result_m["gt_total"] + ground_temp
        result_m["cnt"] = result_m["cnt"] + 1
    else:
        result_m["t_total"] = temp
        result_m["gt_total"] = ground_temp
        result_m["cnt"] = 1

    return result_m


def init(result_m):
    result_m = {}
    return result_m


def data_calling(day, id, month_result):
    startDt, endDt = date_change(day)

    url = "http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList"
    params = {
        "serviceKey": "JblClLk+uLRS6hB9k3LbHV8cJYUxFo3XL4pNRgX5yJOdGf8E8wlFkrB8so5AunCF9QnI/tIdf/vkDbpqcWwzKg==",
        "pageNo": 1,
        "numOfRows": "365",
        "dataType": "XML",
        "dataCd": "ASOS",
        "dateCd": "DAY",
        "startDt": startDt,
        "endDt": endDt,
        "stnIds": id,
    }

    response = requests.get(url, params=params)

    # pprint.pprint(response.text, depth=4)

    soup = BeautifulSoup(response.content, features="xml")
    # pprint.pprint(soup.text)
    items = soup.find_all("item")

    if items is None:
        print("nothing in items")

    month = "01"

    result_m = {}
    # result_m = init(result_m)

    for item in items:
        TM = item.find("tm").get_text()

        months = TM.split("-")

        if month == months[1]:
            result_m = mean_cul(item, result_m)

        else:
            if "error" in result_m:
                count = result_m["cnt"] - result_m["error"]
            else:
                count = result_m["cnt"]

            m_temp = result_m["t_total"] / count
            m_gtemp = result_m["gt_total"] / count
            month_result.append(
                parse_month(
                    months[0],
                    month,
                    result_m["stnid"],
                    result_m["stnnm"],
                    m_temp,
                    m_gtemp,
                )
            )
            month = months[1]
            result_m = {}
            # result_m = init(result_m)
            result_m = mean_cul(item, result_m)

    if "error" in result_m:
        count = result_m["cnt"] - result_m["error"]
    else:
        count = result_m["cnt"]

    m_temp = result_m["t_total"] / count
    m_gtemp = result_m["gt_total"] / count

    month_result.append(
        parse_month(
            months[0], month, result_m["stnid"], result_m["stnnm"], m_temp, m_gtemp
        )
    )

    return month_result


# year_result = []
month_result = []
date_list = [
    "2014-01-01",
    "2015-01-01",
    "2016-01-01",
    "2017-01-01",
    "2018-01-01",
    "2019-01-01",
    "2020-01-01",
    "2021-01-01",
    "2022-01-01",
    "2023-01-01",
]
id_list = opi_middle_list

for date in date_list:
    for id in id_list:
        month_result = data_calling(date, id, month_result)


df = pd.DataFrame(month_result)
print("df pass")
print(df)

# csv 파일로 저장하기
df.to_csv("month_mean.csv", index=False)

KeyError: 'cnt'

In [2]:
import requests
import pandas as pd
from datetime import datetime

# make_params()함수에 사용하는 내부 함수. 날짜를 변경하는 함수
def date_change(date):
    nowdate = datetime.strptime(date, "%Y-%m-%d")
    ldate = (nowdate.replace(month=12).replace(day=31)).strftime("%Y%m%d")
    nowdate_str = nowdate.strftime("%Y%m%d")
    return nowdate_str, ldate

# 변경된 날짜로 파라미터를 생성하는 함수
def make_params(date):
    startDt, endDt = date_change(date)
    result = {
    'serviceKey' : 'fF/Ekuf8awXAsiiMYAivKxzIu1Hyl76PXGGqz88Dvs7sMLoKbSaWzAxF2//kihhIurY9PYr5kxC6lEvw8mDC4Q==',
    'pageNo' : '1',
    'numOfRows' : '365',
    'dataType' : 'XML',
    'dataCd' : 'ASOS',
    'dateCd' : 'DAY',
    'startDt' : startDt,
    'endDt' : endDt,
    'stnIds' : '272'
    }
    return result

# api로 데이터 가져와서 적당히 정제하는 코드
def get_api(url, params):
    response = requests.get(url, params=params)
    contents = response.content
    decode_xml= contents.decode('UTF-8')
    return decode_xml.replace('/','').replace('>','').split('<')

# data_filter() 함수에 사용하는 내부 함수. 빈 문자열 제거하는 함수
def get_data(list,char):
    result = []
    for i in list:
        char_len = len(char)
        if char in i:
            result.append(i[char_len:])
    return [item for item in result if item and item.strip()]

# 원하는 데이터(딕셔너리에 정의된 키:값 사용)만 추출하는 함수
def data_filter(data,dict):
    result = {}
    for a,b in dict.items():
        result = {**result,**{a:get_data(data,b)}}
    return result


url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
dict = {
    '지점번호' : "stnId",
    '지점명' : 'stnNm',
    '시간' : 'tm',
    '평균 기온' : 'avgTa',
    '평균 지면온도' : 'avgTs'
}
date_list = ["2014-01-01"]


for date in date_list:
    params = make_params(date)
    data = get_api(url, params)
    filt_Data = data_filter(data,dict)
    df = pd.DataFrame(filt_Data)
    print(df)

    지점번호 지점명          시간 평균 기온 평균 지면온도
0    272  영주  2014-01-01   4.5     1.4
1    272  영주  2014-01-02   1.9     2.0
2    272  영주  2014-01-03   0.2    -0.6
3    272  영주  2014-01-04   1.9     1.7
4    272  영주  2014-01-05   1.7     2.2
..   ...  ..         ...   ...     ...
360  272  영주  2014-12-27  -3.9    -3.9
361  272  영주  2014-12-28  -2.7    -3.4
362  272  영주  2014-12-29   1.0    -0.4
363  272  영주  2014-12-30   3.3     0.6
364  272  영주  2014-12-31  -1.9    -1.6

[365 rows x 5 columns]


In [4]:
import requests
import pandas as pd
from datetime import datetime, timedelta

def date_change(date):

    nowdate = datetime.strptime(date, "%Y-%m-%d")
    nextYear = nowdate.replace(year=nowdate.year + 1)
    
    endDate = (nextYear - timedelta(days=1)).strftime("%Y%m%d")
    startDate = nowdate.strftime("%Y%m%d")

    return startDate, endDate

def params_settig(date,opi):

    startDt , endDt = date_change(date)
    result = {
        'serviceKey' : 'fF/Ekuf8awXAsiiMYAivKxzIu1Hyl76PXGGqz88Dvs7sMLoKbSaWzAxF2//kihhIurY9PYr5kxC6lEvw8mDC4Q==',
        'pageNo' : '1',
        'numOfRows' : '365',
        'dataType' : 'XML',
        'dataCd' : 'ASOS',
        'dateCd' : 'DAY',
        'startDt' : startDt,
        'endDt' : endDt,
        'stnIds' : opi
    }

    return result

def get_api(url, params):
    response = requests.get(url, params=params)
    contents = response.content
    decode_xml= contents.decode('UTF-8')
    return decode_xml.replace('/','').replace('>','').split('<')

def get_data(list,char):
    result = []
    for i in list:
        char_len = len(char)
        if char in i:
            result.append(i[char_len:])
    return [item for item in result if item and item.strip()]

def data_filter(data,dict):
    result = {}
    for a,b in dict.items():
        result = {**result,**{a:get_data(data,b)}}
    return result

# def valid_opi_list():
#     data = pd.read_csv('./observation_point_info.csv',encoding="CP949")
#     OPI_df = pd.DataFrame(data)
#     observation_point_num_list = [90,93,95,98,99,100,101,102,104,105,106,108,112,114,115,119,121,127,129,130,131,133,135,136,137,138,140,143,146,152,155,156,159,162,165,168,169,170,172,174,177,184,185,188,189,192,201,202,203,211,212,216,217,221,226,232,235,236,238,239,243,244,245,247,248,251,252,253,254,255,257,258,259,260,261,262,263,264,266,268,271,272,273,276,277,278,279,281,283,284,285,288,289,294,295]
    
#     opi2_df = OPI_df[OPI_df['위도']<37]
#     final_opi_df = opi2_df[opi2_df['위도']>36]
#     opi_point_list = list(final_opi_df['지점'])

#     opi_middle_list = []
#     for point in opi_point_list:
#         if point in observation_point_num_list:
#             opi_middle_list.append(point)

#     opi_middle_list = list(set(opi_middle_list))
#     date = "2014-01-01"
#     url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
#     dict = {
#         '지점번호' : "stnId",
#         '지점명' : 'stnNm',
#         '시간' : 'tm',
#         '평균 기온' : 'avgTa',
#         '평균 지면온도' : 'avgTs'
#     }

#     remove_list = []
#     for opi in opi_middle_list:
#         try:
#             params = params_settig(date=date, opi=opi)
#             data = get_api(url,params)
#             filt_data = data_filter(data,dict)
#             df = pd.DataFrame(filt_data)
#             print(df)
#         except:
#             remove_list.append(opi)
    
#     valid_opi_list = [i for i in opi_middle_list if i not in remove_list]
#     return valid_opi_list

In [11]:
def valid_opi_list():
    data = pd.read_csv('./observation_point_info.csv',encoding="CP949")
    OPI_df = pd.DataFrame(data)
    observation_point_num_list = [90,93,95,98,99,100,101,102,104,105,106,108,112,114,115,119,121,127,129,130,131,133,135,136,137,138,140,143,146,152,155,156,159,162,165,168,169,170,172,174,177,184,185,188,189,192,201,202,203,211,212,216,217,221,226,232,235,236,238,239,243,244,245,247,248,251,252,253,254,255,257,258,259,260,261,262,263,264,266,268,271,272,273,276,277,278,279,281,283,284,285,288,289,294,295]
    
    opi2_df = OPI_df[OPI_df['위도']<37]
    final_opi_df = opi2_df[opi2_df['위도']>36]
    opi_point_list = list(set(final_opi_df['지점']))

    opi_middle_list = []
    for point in opi_point_list:
        if point in observation_point_num_list:
            opi_middle_list.append(point)

    date = "2014-01-01"
    url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
    dict = {
        '지점번호' : "stnId",
        '지점명' : 'stnNm',
        '시간' : 'tm',
        '평균 기온' : 'avgTa',
        '평균 지면온도' : 'avgTs'
    }

    remove_list = []
    for opi in opi_middle_list:
        try:
            params = params_settig(date=date, opi=opi)
            data = get_api(url,params)
            filt_data = data_filter(data,dict)
            df = pd.DataFrame(filt_data)
        except:
            remove_list.append(opi)
    
    valid_opi_list = [i for i in opi_middle_list if i not in remove_list]
    return valid_opi_list

In [12]:
valid_opi_list = valid_opi_list()
valid_opi_list

[129,
 130,
 131,
 133,
 135,
 136,
 137,
 138,
 140,
 272,
 273,
 276,
 277,
 278,
 279,
 177,
 226,
 232,
 235,
 236,
 238,
 239,
 127]

In [13]:
len(valid_opi_list)

23

In [7]:
valid_opi_list = [129, 130, 131, 133, 135, 136, 137, 138, 140, 271, 272, 273, 276, 277, 278, 279, 177, 226, 232, 235, 236, 238, 239, 127]

In [8]:
len(valid_opi_list)

24

In [14]:
date_list = ["2014-01-01","2015-01-01","2016-01-01","2017-01-01","2018-01-01","2019-01-01","2020-01-01","2021-01-01","2022-01-01","2023-01-01"]
url = 'http://apis.data.go.kr/1360000/AsosDalyInfoService/getWthrDataList'
dict = {
    '지점번호' : "stnId",
    '지점명' : 'stnNm',
    '시간' : 'tm',
    '평균 기온' : 'avgTa',
    '평균 지면온도' : 'avgTs'
}

for date in date_list:
    for opn in valid_opi_list:
        params = params_settig(date=date, opi=opn)
        data = get_api(url,params)
        filt_data = data_filter(data,dict)
        df = pd.DataFrame(filt_data)
        print(df)

    지점번호 지점명          시간 평균 기온 평균 지면온도
0    129  서산  2014-01-01   3.0     1.8
1    129  서산  2014-01-02   0.7     1.8
2    129  서산  2014-01-03   2.3     2.1
3    129  서산  2014-01-04  -1.1     1.4
4    129  서산  2014-01-05  -2.0     0.7
..   ...  ..         ...   ...     ...
360  129  서산  2014-12-27  -1.8     0.8
361  129  서산  2014-12-28  -0.7     0.4
362  129  서산  2014-12-29   1.3     1.3
363  129  서산  2014-12-30   1.4     1.4
364  129  서산  2014-12-31  -0.5     1.2

[365 rows x 5 columns]
    지점번호 지점명          시간 평균 기온 평균 지면온도
0    130  울진  2014-01-01   5.2     1.8
1    130  울진  2014-01-02   3.2     1.1
2    130  울진  2014-01-03   5.3     1.6
3    130  울진  2014-01-04   0.2    -0.3
4    130  울진  2014-01-05   1.8     0.9
..   ...  ..         ...   ...     ...
360  130  울진  2014-12-27   2.8     0.4
361  130  울진  2014-12-28   4.0     1.8
362  130  울진  2014-12-29   4.5     2.1
363  130  울진  2014-12-30   5.8     2.5
364  130  울진  2014-12-31   2.5     1.6

[365 rows x 5 columns]
    지점번호 지점명    

ValueError: All arrays must be of the same length